In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
import optuna
import random
import gc
import json


import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, ReLU, BatchNormalization
from tensorflow.keras.initializers import HeNormal, Ones, Constant
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import display
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [2]:
def set_seed(seed_value=3126):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

SEED = 3126
set_seed()

In [3]:
WIDTH, HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 12
DATA_PATH = '/kaggle/input/finalized-astrovision-data'

#data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATA_PATH,
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED  
)

validation_generator = train_datagen.flow_from_directory(
    DATA_PATH,
    target_size=(WIDTH, HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED 
)

Found 4913 images belonging to 4 classes.
Found 1228 images belonging to 4 classes.


In [4]:
def make_dense_layer(input_size, dropout_rate=0.0, input_shape=None):
    layers = []
    if input_shape:
        layers.append(Dense(input_size,
                            use_bias=False, 
                            input_shape=input_shape,
                            kernel_initializer=HeNormal()
                            ))
    else:
        layers.append(Dense(input_size, use_bias=False, kernel_initializer=HeNormal()))
    layers.extend([BatchNormalization(gamma_initializer=Ones(), beta_initializer=Constant(0.25)),
                  ReLU()])
    if dropout_rate > 0:
        layers.append(Dropout(dropout_rate))
    return Sequential(layers)

In [5]:
best_weights_path = "/kaggle/working/best_weights.weights.h5"

early_stopping_callback = EarlyStopping(
    monitor='val_loss',  
    patience=10,              
    mode='min',              
    min_delta=.001,
    restore_best_weights=True,  
    verbose=1
)

In [6]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        K.clear_session()  
        gc.collect() 

In [7]:
best_val_acc = 0

def base_model_objective(trial):
    global best_val_acc
    ## Define distributions of hyperparams
    learning_rate_ = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    weight_decay_  = trial.suggest_float("weight_decay", 1e-12, 1e-1, log=True)
    beta1_         = trial.suggest_float("beta1", .895, .999) 
    beta2_         = trial.suggest_float("beta2", .895, .999) 

    # Build the model
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(WIDTH, HEIGHT, 3))
    base_model.trainable = False  # Freeze base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    fc_layers = Sequential([
        make_dense_layer(1024, input_shape=(x.shape[-1],)),
        make_dense_layer(512),
        make_dense_layer(8)
    ])
    x = fc_layers(x)
    predictions = Dense(train_generator.num_classes, activation='softmax', kernel_initializer=HeNormal())(x)

    checkpoint_callback = ModelCheckpoint(
        filepath=best_weights_path,  
        monitor='val_accuracy',    
        save_best_only=True,       
        save_weights_only=True,    
        mode='max',                
        verbose=1,
        initial_value_threshold=best_val_acc
    )
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=AdamW(learning_rate_, weight_decay_, beta1_, beta2_),
                  loss='categorical_crossentropy', metrics=['accuracy'])

    # Reload pretrained weights (if necessary)
    model.load_weights("/kaggle/input/baseline-tf-model-3-hidden/best_weights.weights.h5")

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks=[checkpoint_callback, early_stopping_callback, ClearMemory()]
    )

    loss, val_acc = model.evaluate(validation_generator, 
                                   steps=validation_generator.samples // BATCH_SIZE)
    
    if(val_acc>best_val_acc):
        best_val_acc = val_acc
        print(f"\nNew best model found with val_acc = {best_val_acc:.4f}\n")
    return val_acc

In [8]:
## Objective is to maximize accuracy!
study = optuna.create_study(study_name="baseline_params_search", direction="maximize")
study.optimize(base_model_objective, n_trials=22)

[I 2025-03-10 19:22:05,594] A new study created in memory with name: baseline_params_search


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 593ms/step - accuracy: 0.8181 - loss: 0.4011
Epoch 1: val_accuracy improved from 0.00000 to 0.78502, saving model to /kaggle/working/best_weights.weights.h5
154/154 ━━━━━━━━━━━━━━━━━━━━ 136s 784ms/step - accuracy: 0.8181 - loss: 0.4012 - val_accuracy: 0.7850 - val_loss: 0.4454
Epoch 2/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - accuracy: 0.8271 - loss: 0.3899
Epoch 2: val_accuracy improved from 0.78502 to 0.80456, saving model to /kaggle/working/best_weights.weights.h5
154/154 ━━━━━━━━━━━━━━━━━━━━ 75s 464ms/step - accuracy: 0.8270 - loss: 0.3900 - val_accuracy: 0.8046 - val_loss: 0.4615
Epoch 3/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.8233 - loss: 0.4032
Epoch 3: val_accuracy did not improve from 0.80456
154/154 ━━━━━━━━━━━━━━━━━━━━ 74s 461ms/step - accuracy: 0.8233 - loss: 0.4031 - val_accuracy: 0.7964 - val_loss: 0.4205
Epoch 4/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/st

[I 2025-03-10 19:38:23,643] Trial 0 finished with value: 0.8223684430122375 and parameters: {'learning_rate': 0.006714819991380839, 'weight_decay': 1.567225612893073e-10, 'beta1': 0.9198385739502527, 'beta2': 0.95248078116008}. Best is trial 0 with value: 0.8223684430122375.



New best model found with val_acc = 0.8224

Epoch 1/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.8262 - loss: 0.4006
Epoch 1: val_accuracy improved from 0.82237 to 0.82818, saving model to /kaggle/working/best_weights.weights.h5
154/154 ━━━━━━━━━━━━━━━━━━━━ 88s 502ms/step - accuracy: 0.8263 - loss: 0.4004 - val_accuracy: 0.8282 - val_loss: 0.3683
Epoch 2/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.8422 - loss: 0.3593
Epoch 2: val_accuracy did not improve from 0.82818
154/154 ━━━━━━━━━━━━━━━━━━━━ 74s 458ms/step - accuracy: 0.8423 - loss: 0.3592 - val_accuracy: 0.8111 - val_loss: 0.3910
Epoch 3/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.8475 - loss: 0.3595
Epoch 3: val_accuracy improved from 0.82818 to 0.82980, saving model to /kaggle/working/best_weights.weights.h5
154/154 ━━━━━━━━━━━━━━━━━━━━ 74s 461ms/step - accuracy: 0.8474 - loss: 0.3595 - val_accuracy: 0.8298 - val_loss: 0.3725
Epoch 4/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step - 

[I 2025-03-10 19:51:20,407] Trial 1 finished with value: 0.8322368264198303 and parameters: {'learning_rate': 0.0005735247287532914, 'weight_decay': 0.06057478524282291, 'beta1': 0.9411660322497772, 'beta2': 0.9414513257861091}. Best is trial 1 with value: 0.8322368264198303.


...

[I 2025-03-10 23:55:10,529] Trial 19 finished with value: 0.8396381735801697 and parameters: {'learning_rate': 2.4661242734821514e-05, 'weight_decay': 9.801096390989685e-07, 'beta1': 0.9612633407099205, 'beta2': 0.9130825137368326}. Best is trial 12 with value: 0.8544408082962036.


Epoch 1/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.8269 - loss: 0.3866
Epoch 1: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 90s 516ms/step - accuracy: 0.8269 - loss: 0.3867 - val_accuracy: 0.8412 - val_loss: 0.3651
Epoch 2/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - accuracy: 0.8186 - loss: 0.3966
Epoch 2: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 76s 473ms/step - accuracy: 0.8187 - loss: 0.3966 - val_accuracy: 0.8249 - val_loss: 0.3746
Epoch 3/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.8157 - loss: 0.4008
Epoch 3: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 76s 470ms/step - accuracy: 0.8159 - loss: 0.4006 - val_accuracy: 0.8331 - val_loss: 0.3803
Epoch 4/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.8252 - loss: 0.3804
Epoch 4: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 76s 471ms/step - accuracy: 0.8251 - loss: 0.3805 - val_

[I 2025-03-11 00:10:53,434] Trial 20 finished with value: 0.8445723652839661 and parameters: {'learning_rate': 1.0051322397160368e-05, 'weight_decay': 1.6291027268457878e-05, 'beta1': 0.9220212306586055, 'beta2': 0.9622251429242576}. Best is trial 12 with value: 0.8544408082962036.


Epoch 1/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step - accuracy: 0.8218 - loss: 0.4002
Epoch 1: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 89s 513ms/step - accuracy: 0.8218 - loss: 0.4002 - val_accuracy: 0.8347 - val_loss: 0.3853
Epoch 2/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step - accuracy: 0.8333 - loss: 0.3788
Epoch 2: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 77s 480ms/step - accuracy: 0.8332 - loss: 0.3789 - val_accuracy: 0.8428 - val_loss: 0.3820
Epoch 3/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - accuracy: 0.8187 - loss: 0.3998
Epoch 3: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 76s 474ms/step - accuracy: 0.8188 - loss: 0.3996 - val_accuracy: 0.8453 - val_loss: 0.3669
Epoch 4/12
153/154 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - accuracy: 0.8372 - loss: 0.3813
Epoch 4: val_accuracy did not improve from 0.85444
154/154 ━━━━━━━━━━━━━━━━━━━━ 76s 473ms/step - accuracy: 0.8372 - loss: 0.3814 - val_

[I 2025-03-11 00:24:09,021] Trial 21 finished with value: 0.8371710777282715 and parameters: {'learning_rate': 1.0758335626265775e-05, 'weight_decay': 0.0018317744691792582, 'beta1': 0.9035405849032465, 'beta2': 0.9231423738710876}. Best is trial 12 with value: 0.8544408082962036.


In [9]:
print("Best parameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

Best parameters: {'learning_rate': 1.0954700275088106e-05, 'weight_decay': 0.00014313277277334192, 'beta1': 0.8956561164123156, 'beta2': 0.9224535952932861}
Best validation accuracy: 0.8544408082962036
